In [1]:
!pip3 install qiskit qiskit-machine-learning torch numpy scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 16.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Qiskit & QML imports
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

# Fix random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

### 1. Create toy dataset (binary classification, 2 features)
X, y = make_blobs(n_samples=200, centers=2, n_features=2, cluster_std=1.5, random_state=42)
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test,  dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test  = torch.tensor(y_test,  dtype=torch.long)

### 2. Build quantum circuit (feature map + ansatz)
feature_map = ZZFeatureMap(feature_dimension=2, reps=1)  # encodes data
ansatz = RealAmplitudes(num_qubits=2, reps=1)           # variational circuit

qc = feature_map.compose(ansatz)

# QNN setup: map 2-D classical input → expectation value(s)
qnn = EstimatorQNN(
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    # you can set output shape etc., for simple classification -> scalar output
)

# Wrap as a PyTorch module
qnn_torch = TorchConnector(qnn)

class HybridClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.quantum_layer = qnn_torch
        self.classifier = nn.Linear(1, 2)  # quantum output → 2 classes

    def forward(self, x):
        qout = self.quantum_layer(x)         # shape: [batch, 1]
        logits = self.classifier(qout)       # shape: [batch, 2]
        return logits

model = HybridClassifier()

### 3. Training setup
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()
epochs = 20

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    logits = model(X_train)
    loss = loss_fn(logits, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0 or epoch == 0:
        with torch.no_grad():
            pred = torch.argmax(model(X_test), dim=1)
            acc = (pred == y_test).float().mean()
        print(f"Epoch {epoch+1:2d}: loss = {loss.item():.4f}, test_acc = {acc:.4f}")

### 4. Final evaluation
model.eval()
with torch.no_grad():
    pred = torch.argmax(model(X_test), dim=1)
    acc = (pred == y_test).float().mean()
print("Final test accuracy:", acc.item())

/tmp/ipython-input-1819465598.py:36: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch  1: loss = 0.8192, test_acc = 0.4250
Epoch  5: loss = 0.7976, test_acc = 0.4250
Epoch 10: loss = 0.7745, test_acc = 0.4250
Epoch 15: loss = 0.7554, test_acc = 0.4250
Epoch 20: loss = 0.7398, test_acc = 0.4250
Final test accuracy: 0.42500001192092896


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Qiskit & QML imports
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

# Fix random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

### 1. Create toy dataset (binary classification, 2 features)
X, y = make_blobs(n_samples=200, centers=2, n_features=2, cluster_std=1.5, random_state=42)
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test,  dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test  = torch.tensor(y_test,  dtype=torch.long)

### 2. Build quantum circuit (feature map + ansatz)
feature_map = ZZFeatureMap(feature_dimension=2, reps=1)  # encodes data
ansatz = RealAmplitudes(num_qubits=2, reps=1)           # variational circuit

qc = feature_map.compose(ansatz)

# QNN setup: map 2-D classical input → expectation value(s)
qnn = EstimatorQNN(
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    # you can set output shape etc., for simple classification -> scalar output
)

# Wrap as a PyTorch module
qnn_torch = TorchConnector(qnn)

class HybridClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.quantum_layer = qnn_torch
        self.classifier = nn.Linear(1, 2)  # quantum output → 2 classes

    def forward(self, x):
        qout = self.quantum_layer(x)         # shape: [batch, 1]
        logits = self.classifier(qout)       # shape: [batch, 2]
        return logits

model = HybridClassifier()

### 3. Training setup
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()
epochs = 20

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    logits = model(X_train)
    loss = loss_fn(logits, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0 or epoch == 0:
        with torch.no_grad():
            pred = torch.argmax(model(X_test), dim=1)
            acc = (pred == y_test).float().mean()
        print(f"Epoch {epoch+1:2d}: loss = {loss.item():.4f}, test_acc = {acc:.4f}")

### 4. Final evaluation
model.eval()
with torch.no_grad():
    pred = torch.argmax(model(X_test), dim=1)
    acc = (pred == y_test).float().mean()
print("Final test accuracy:", acc.item())

/tmp/ipython-input-1819465598.py:36: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch  1: loss = 0.8192, test_acc = 0.4250
Epoch  5: loss = 0.7976, test_acc = 0.4250


KeyboardInterrupt: 

In [5]:
!pip install qiskit-aer



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 81.9 MB/s eta 0:00:00
=== Noise setting: Ideal / No noise


TypeError: EstimatorQNN.__init__() got an unexpected keyword argument 'backend'

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error

# reproducibility
np.random.seed(0)
torch.manual_seed(0)

### 1. Create toy dataset (binary classification, 2-D data)
X, y = make_blobs(n_samples=200, centers=2, n_features=2,
                  cluster_std=1.5, random_state=0)
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test,  dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test  = torch.tensor(y_test, dtype=torch.long)

### 2. Build quantum circuit (feature map + ansatz)
feature_map = ZZFeatureMap(feature_dimension=2, reps=1)
ansatz = RealAmplitudes(num_qubits=2, reps=1)
qc = feature_map.compose(ansatz)

### Helper: build a Torch-compatible hybrid model given an Aer simulator
def make_model(aer_backend):
    # Create an Estimator primitive that uses this simulator
    estimator = Estimator(options={"backend": aer_backend})
    # Build QNN with that estimator
    qnn = EstimatorQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        estimator=estimator
    )
    qnn_torch = TorchConnector(qnn)

    class HybridClassifier(nn.Module):
        def __init__(self):
            super().__init__()
            self.quantum = qnn_torch
            self.classifier = nn.Linear(1, 2)

        def forward(self, x):
            q_out = self.quantum(x)          # shape [batch, 1]
            logits = self.classifier(q_out)  # shape [batch, 2]
            return logits

    return HybridClassifier()

### Helper: build a custom noise model
def get_noise_model(p1q=0.0, p2q=0.0):
    nm = NoiseModel()
    if p1q > 0:
        err1 = depolarizing_error(p1q, 1)
        nm.add_all_qubit_quantum_error(err1, ['rz', 'sx', 'x', 'ry', 'u1', 'u2', 'u3'])
    if p2q > 0:
        err2 = depolarizing_error(p2q, 2)
        nm.add_all_qubit_quantum_error(err2, ['cx'])
    return nm

### Experiment: run with various noise levels
noise_settings = [
    (0.0, 0.0, "Ideal / No noise"),
    (0.005, 0.005, "Low noise"),
    (0.01, 0.02,  "Medium noise"),
    (0.02, 0.05,  "High noise"),
]

for p1, p2, label in noise_settings:
    print("\n=== Noise setting:", label)
    nm = get_noise_model(p1q=p1, p2q=p2)
    aer = AerSimulator(method='density_matrix', noise_model=nm)

    # Build model for this simulator
    model = make_model(aer)

    optimizer = optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    # Train a bit
    for epoch in range(1000):
        optimizer.zero_grad()
        logits = model(X_train)
        loss = criterion(logits, y_train)
        loss.backward()
        optimizer.step()

    # Evaluate
    model.eval()
    with torch.no_grad():
        logits = model(X_test)
        loss = criterion(logits, y_test)
        pred = torch.argmax(logits, dim=1)
        acc = (pred == y_test).float().mean()
    print(f"Test Loss: {loss.item():.4f}, Test Acc: {acc:.4f}")


=== Noise setting: Ideal / No noise


/tmp/ipython-input-3331130768.py:41: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator(options={"backend": aer_backend})
/tmp/ipython-input-3331130768.py:43: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Test Loss: 0.7789, Test Acc: 0.4250

=== Noise setting: Low noise
Test Loss: 0.7789, Test Acc: 0.4250

=== Noise setting: Medium noise
Test Loss: 0.7789, Test Acc: 0.4250

=== Noise setting: High noise
Test Loss: 0.7789, Test Acc: 0.4250
